### SVR
* grid search
* 4-folded cv

In [7]:
#polynomial n sigmoid kernel
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error
from tools import create_vectors
import pandas as pd

In [6]:
data = pd.read_csv('../datasets/data_binned.csv', parse_dates=['date_'])

In [ ]:
x, y = create_vectors(data, ['std_pos'])

In [ ]:
def create_and_predict(x, y):
    tscv = TimeSeriesSplit(n_splits=6)
    regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, cache_size=1000))
    vals = cross_val_score(regr, x, y, scoring='neg_mean_absolute_percentage_error', cv=tscv)
    return sum(vals)/len(vals) * -1

In [ ]:
cols = ['open', 'high', 'low', 'vol', 'max_comp',
       'max_neg', 'max_neu', 'max_pos', 'min_comp', 'min_neg', 'min_neu',
       'min_pos', 'std_comp', 'std_neg', 'std_neu', 'std_pos', 'mean_comp',
       'mean_neg', 'mean_neu', 'mean_pos', 'median_comp', 'median_neg',
       'median_neu', 'median_pos', 'count']
lags = [2, 3, 5, 6, 7, 8, 10, 12, 16]

error = {i:{} for i in lags}
for lag in lags:
    x, y = create_vectors(data, lag=lag+1)
    mape = create_and_predict(x, y)
    for col in cols:
        x, y = create_vectors(data, [col])
        er = create_and_predict(x, y)  
        error[lag][col] = (round(er, 3), "Poprawa" if er < mape else "Gorzej")


In [ ]:
for i in error:
    dict(sorted(error[i].items(), key=lambda item: item[1]))

In [ ]:
error[3].items()[1]

TypeError: 'dict_items' object is not subscriptable

In [ ]:

error_bak = error

for lag in lags:
    for i, j  in error[lag].copy().items():
        if j[1] == 'Gorzej':
            error[lag].pop(i)

In [ ]:
error